# Accent Detection 

In [ ]:
import pyaudio

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Open a stream for real-time audio capture
stream = audio.open(format=pyaudio.paInt16, 
                    channels=1,
                    rate=16000,
                    input=True,
                    frames_per_buffer=1024)

print("Recording...")
while True:
    data = stream.read(1024)
    # Process audio data here


In [ ]:
import openai

# Set your API key
openai.api_key = "your_openai_api_key"

# Transcribe audio using OpenAI API
audio_file = open("media/recorded_audio.wav", "rb")
transcription = openai.Audio.transcribe("whisper-1", audio_file)
print("Transcription:", transcription["text"])


In [6]:
import pyaudio
from google.cloud import speech

def real_time_transcription():
    """Performs real-time speech-to-text transcription."""
    client = speech.SpeechClient()

    # Configure audio settings
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )
    streaming_config = speech.StreamingRecognitionConfig(config=config, interim_results=True)

    def generator():
        """Generator that yields audio chunks from the microphone."""
        audio = pyaudio.PyAudio()
        stream = audio.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=1024)

        try:
            while True:
                yield speech.StreamingRecognizeRequest(audio_content=stream.read(1024))
        except GeneratorExit:
            stream.stop_stream()
            stream.close()
            audio.terminate()

    # Perform streaming recognition
    requests = generator()
    responses = client.streaming_recognize(config=streaming_config, requests=requests)

    # Print transcriptions
    for response in responses:
        for result in response.results:
            if result.is_final:
                print(f"Transcription: {result.alternatives[0].transcript}")

# Call the function
real_time_transcription()


Transcription: 
Transcription: how much time
Transcription:  hello can you hear me
Transcription:  okay stop listening and give me the output
Transcription:  it's perfect right
Transcription:  so what do you think about it


KeyboardInterrupt: 

In [12]:
import pyaudio
import wave
from google.cloud import speech
from phonemizer import phonemize

# Configure PyAudio
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "user_input.wav"

def record_audio():
    """Record audio from the microphone."""
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

    print("Recording...")
    frames = []
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording stopped.")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded audio
    with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

def transcribe_audio():
    """Transcribe audio using Google Speech-to-Text."""
    client = speech.SpeechClient()
    with open(WAVE_OUTPUT_FILENAME, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="en-US",
    )

    response = client.recognize(config=config, audio=audio)
    if response.results:
        return response.results[0].alternatives[0].transcript
    else:
        return None

def compare_pronunciations(correct_word, user_input, language="en-gb"):
    """Compare the pronunciation of a correct word with user input."""
    correct_pronunciation = phonemize(correct_word, language=language, backend="espeak", strip=True)
    user_pronunciation = phonemize(user_input, language=language, backend="espeak", strip=True)

    if correct_pronunciation == user_pronunciation:
        print("Correct pronunciation!")
    else:
        print(f"Pronunciation mismatch:\n"
              f"  Expected: {correct_pronunciation}\n"
              f"  Got: {user_pronunciation}")

# Main flow
if __name__ == "__main__":
    record_audio()
    print("Transcribing audio...")
    transcribed_text = transcribe_audio()
    if transcribed_text:
        print(f"Transcribed text: {transcribed_text}")
        compare_pronunciations("example", transcribed_text)
    else:
        print("No transcription available.")


ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
Jac

Recording...
Recording stopped.
Transcribing audio...
No transcription available.


In [9]:
# Install pyttsx3 if not already installed
%pip install pyttsx3
%pip install gtts

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyaudio
import wave
from google.cloud import speech
from phonemizer import phonemize
from gtts import gTTS
import os
import shutil

# Configure PyAudio
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "user_input.wav"
TTS_OUTPUT_FILENAME = "correct_pronunciation.mp3"

# Ensure mpg321 or another MP3 player is installed
def play_audio(file_path):
    """Play audio file using an OS-specific command."""
    if shutil.which("mpg321"):
        os.system(f"mpg321 {file_path}")
    elif shutil.which("ffplay"):
        os.system(f"ffplay -nodisp -autoexit {file_path}")
    elif shutil.which("afplay"):
        os.system(f"afplay {file_path}")
    else:
        print(f"Error: MP3 player not found. Install mpg321, ffplay, or afplay to play {file_path}.")


def record_audio():
    """Record audio from the microphone."""
    audio = pyaudio.PyAudio()
    try:
        stream = audio.open(format=FORMAT,
                            channels=CHANNELS,
                            rate=RATE,
                            input=True,
                            frames_per_buffer=CHUNK)

        print("Recording...")
        frames = []
        for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

        print("Recording stopped.")

        # Save the recorded audio
        with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(audio.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(b''.join(frames))
    finally:
        stream.stop_stream()
        stream.close()
        audio.terminate()

def transcribe_audio():
    """Transcribe audio using Google Speech-to-Text."""
    client = speech.SpeechClient()
    with open(WAVE_OUTPUT_FILENAME, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="en-US",
    )

    response = client.recognize(config=config, audio=audio)
    if response.results:
        return response.results[0].alternatives[0].transcript
    else:
        return None

def compare_pronunciation(correct, user):
    """Compare the pronunciation of the correct and user input."""
    errors = []
    for c, u in zip(correct.split(), user.split()):
        if c != u:
            errors.append(f"Pronounced '{u}' instead of '{c}'")
    return errors

def text_to_speech(correct_pronunciation):
    """Use Google TTS to guide the user on correct pronunciation."""
    tts = gTTS(f"Try pronouncing it as: {correct_pronunciation}", lang="en")
    tts.save(TTS_OUTPUT_FILENAME)
    play_audio(TTS_OUTPUT_FILENAME)

# Main flow
if __name__ == "__main__":
    record_audio()
    print("Transcribing audio...")
    transcribed_text = transcribe_audio()

    if transcribed_text:
        print(f"Transcribed text: {transcribed_text}")

        correct_word = "My name is khan"
        correct_pronunciation = phonemize(correct_word, language="en-gb", backend="espeak", strip=True)
        user_pronunciation = phonemize(transcribed_text, language="en-gb", backend="espeak", strip=True)

        # Compare phonemes
        if correct_pronunciation == user_pronunciation:
            print("Correct pronunciation!")
        else:
            errors = compare_pronunciation(correct_word, transcribed_text)
            print("Pronunciation mismatches:")
            for error in errors:
                print(f"  - {error}")

            # Guide the user with TTS
            text_to_speech(correct_pronunciation)
    else:
        print("No transcription available.")


ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
Jac

Recording...
Recording stopped.
Transcribing audio...
No transcription available.


# Implementation Steps
### Step 1: Speech Recognition

Use Whisper API to convert speech to text.

In [2]:
from pynput import keyboard
import pyaudio
import wave
import os
import whisper

# Configuration
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
OUTPUT_FOLDER = "media"
OUTPUT_FILENAME = "recorded_audio.wav"

# Create the output folder if it doesn't exist
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
output_path = os.path.join(OUTPUT_FOLDER, OUTPUT_FILENAME)

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Open a stream for real-time audio capture
stream = audio.open(format=FORMAT, 
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)


print("Recording... Press 'S' to stop.")

frames = []
recording = True

# Function to handle key presses
def on_press(key):
    global recording
    try:
        if key.char == 's':  # Stop recording when 'S' is pressed
            print("\n'S' key pressed. Stopping recording...")
            recording = False
            return False  # Stop the listener
    except AttributeError:
        pass

# Start recording
with keyboard.Listener(on_press=on_press) as listener:
    while recording:
        data = stream.read(CHUNK)
        frames.append(data)
    listener.join()

# Stop and close the stream
stream.stop_stream()
stream.close()
audio.terminate()

# Save the recorded data to a WAV file
with wave.open(output_path, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))

print(f"Audio file saved at: {output_path}")

# Load the Whisper model
print("Loading Whisper model...")
model = whisper.load_model("base")  # Use "base", "small", "medium", or "large"

# Transcribe the audio
print("Transcribing audio...")
result = model.transcribe(output_path)
print("Transcription:")
print(result["text"])

ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
Jac

Recording... Press 'S' to stop.

'S' key pressed. Stopping recording...
Audio file saved at: media/recorded_audio.wav
Loading Whisper model...


/home/vai/Desktop/chat_bot_experiment_one/myenv/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f

Transcribing audio...


/home/vai/Desktop/chat_bot_experiment_one/myenv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:
 Hello, my name is Karen and I am from India.


#### Detect phonemes and linguistic features for accent analysis.

In [1]:
%pip install pronouncing

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 380.0 kB/s eta 0:00:00a 0:00:01
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6292 sha256=afdaebdb767b19befe7528f5bf8d34a92b76b77d0d5eed13bc77ea4ebc2d03fd
  Stored in directory: /home/vai/.cache/pip/wheels/a0/76/15/dfdf38731993cdc4e86fd6d949c70c0e9786cf00073d8114d4
Successfully built pronouncing

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pronouncing

text = "How are you doing today?"
words = text.split()
phonemes = [pronouncing.phones_for_word(word)[0] for word in words if pronouncing.phones_for_word(word)]
print(phonemes)


['HH AW1', 'AA1 R', 'Y UW1', 'D UW1 IH0 NG']


In [2]:
%pip install TTS


ERROR: Ignored the following versions that require a different python version: 0.0.10.2 Requires-Python >=3.6.0, <3.9; 0.0.10.3 Requires-Python >=3.6.0, <3.9; 0.0.11 Requires-Python >=3.6.0, <3.9; 0.0.12 Requires-Python >=3.6.0, <3.9; 0.0.13.1 Requires-Python >=3.6.0, <3.9; 0.0.13.2 Requires-Python >=3.6.0, <3.9; 0.0.14.1 Requires-Python >=3.6.0, <3.9; 0.0.15 Requires-Python >=3.6.0, <3.9; 0.0.15.1 Requires-Python >=3.6.0, <3.9; 0.0.9 Requires-Python >=3.6.0, <3.9; 0.0.9.1 Requires-Python >=3.6.0, <3.9; 0.0.9.2 Requires-Python >=3.6.0, <3.9; 0.0.9a10 Requires-Python >=3.6.0, <3.9; 0.0.9a9 Requires-Python >=3.6.0, <3.9; 0.1.0 Requires-Python >=3.6.0, <3.10; 0.1.1 Requires-Python >=3.6.0, <3.10; 0.1.2 Requires-Python >=3.6.0, <3.10; 0.1.3 Requires-Python >=3.6.0, <3.10; 0.10.0 Requires-Python >=3.7.0, <3.11; 0.10.1 Requires-Python >=3.7.0, <3.11; 0.10.2 Requires-Python >=3.7.0, <3.11; 0.11.0 Requires-Python >=3.7.0, <3.11; 0.11.1 Requires-Python >=3.7.0, <3.11; 0.12.0 Requires-Python >=3

In [15]:
import os
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer
import pronouncing

# Extract phonemes using pronouncing
def extract_phonemes(sentence):
    words = sentence.split()
    phonemes = [pronouncing.phones_for_word(word)[0] for word in words if pronouncing.phones_for_word(word)]
    return phonemes

# Load the Wav2Vec2 processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model is running on: {device}")


Model is running on: cuda


In [ ]:
import numpy as np
import torch
import torchaudio
from datasets import load_dataset
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

# Load processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

# Set a reasonable model_max_length
processor.tokenizer.model_max_length = 512  # Adjust as needed



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model is running on: {device}")

# Preprocessing function
def preprocess_audio(batch):
    try:
        # Step 1: Load audio and validate
        audio = np.array(batch["audio"]["array"], dtype=np.float32)
        sampling_rate = batch["audio"]["sampling_rate"]
        print(f"Audio Length: {len(audio)}, Sampling Rate: {sampling_rate}")

        # Step 2: Resample if necessary
        if sampling_rate != 16000:
            print("Resampling audio to 16kHz")
            resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
            audio = resampler(torch.tensor(audio)).numpy()

        # Step 3: Truncate or pad audio to a fixed length
        max_audio_length = 16000
        if len(audio) > max_audio_length:
            print(f"Truncating audio from {len(audio)} to {max_audio_length}")
            audio = audio[:max_audio_length]
        else:
            # Pad with zeros if shorter
            audio = np.pad(audio, (0, max_audio_length - len(audio)), "constant")

        # Normalize audio
        audio = (audio - np.mean(audio)) / (np.std(audio) + 1e-7)

        # Step 4: Convert text to tokens
        phonemes = batch["sentence"].split()  # Replace with phoneme logic if available
        print(f"Phonemes: {phonemes}")

        # Tokenize phonemes with additional debugging
        try:
            phoneme_ids = processor.tokenizer(
                " ".join(phonemes),
                return_tensors="np",
                padding="max_length",
                truncation=True,
                max_length=processor.tokenizer.model_max_length,
            ).input_ids[0].astype(np.int32)  # Force smaller int32 type

            print(f"Phoneme IDs length: {len(phoneme_ids)}")
        except Exception as e:
            print(f"Error during tokenization: {e}")
            raise ValueError(f"Tokenization failed for text: {' '.join(phonemes)}")

        # Step 5: Process audio for Wav2Vec2
        input_values = processor(
            audio,
            sampling_rate=16000,
            return_tensors="np",
            padding="max_length",
            truncation=True,
            max_length=max_audio_length,
        ).input_values[0]

        # Debugging shapes
        print(f"Processed input_values shape: {input_values.shape}")
        print(f"Processed phoneme_ids shape: {phoneme_ids.shape}")

        # Assign processed data to batch
        batch["input_values"] = input_values
        batch["labels"] = phoneme_ids

    except Exception as e:
        print(f"Error processing batch: {e}")
        raise e

    return batch

# Load dataset
data = load_dataset("DTU54DL/common-accent")

# Preprocess dataset
try:
    data = data.map(preprocess_audio, remove_columns=["audio", "sentence"], batched=False)
except Exception as e:
    print(f"Dataset preprocessing failed: {e}")
    raise e

# Debug preprocessed data
print(data["train"][0])

# Define metric computation
def compute_metrics(pred):
    predictions = torch.argmax(torch.tensor(pred.predictions), axis=-1)
    label_ids = torch.tensor(pred.label_ids)
    accuracy = (predictions == label_ids).float().mean().item()
    return {"accuracy": accuracy}

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=processor)

# Training arguments
training_args = TrainingArguments(
    output_dir="./wav2vec2_common_accent_finetuned",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust based on GPU memory
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train model
try:
    trainer.train()
except Exception as e:
    print(f"Training failed: {e}")
    raise e

# Save model
trainer.save_model("./wav2vec2_common_accent_finetuned")
processor.save_pretrained("./wav2vec2_common_accent_finetuned")


In [28]:
print(data["train"][0])


{'audio': {'path': 'common_voice_en_101616.mp3', 'array': array([-3.78349796e-10, -3.49245965e-10, -2.61934474e-10, ...,
       -1.45020604e-05,  5.48214593e-05,  4.17054107e-05]), 'sampling_rate': 16000}, 'sentence': 'Men in orange vests are at work on a construction site.', 'accent': 'India and South Asia (India, Pakistan, Sri Lanka)'}


In [32]:
print(processor.tokenizer)


Wav2Vec2CTCTokenizer(name_or_path='facebook/wav2vec2-base-960h', vocab_size=32, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	1: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	2: AddedToken("</s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	3: AddedToken("<unk>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
}
)


In [ ]:
import numpy as np
import torch
import torchaudio
from datasets import load_dataset
from TTS.tts.utils.text.phonemizers import get_phonemizer_by_name
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

# Initialize ConquiTTS phonemizer
phonemizer = get_phonemizer_by_name(name="espeak", language="en-us", phoneme_cache_path="./phoneme_cache")
phonemizer.set_gpu(True)  # Enable GPU support for phonemizer

# Load Wav2Vec2 processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

# Set a reasonable model_max_length
processor.tokenizer.model_max_length = 512  # Cap tokenized length

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model is running on: {device}")

# Preprocessing function
def preprocess_audio(batch):
    try:
        # Step 1: Load audio and validate
        audio = np.array(batch["audio"]["array"], dtype=np.float32)
        sampling_rate = batch["audio"]["sampling_rate"]
        print(f"Audio Length: {len(audio)}, Sampling Rate: {sampling_rate}")

        # Step 2: Resample if necessary
        if sampling_rate != 16000:
            print("Resampling audio to 16kHz")
            resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
            audio = resampler(torch.tensor(audio)).numpy()

        # Step 3: Truncate or pad audio to a fixed length
        max_audio_length = 16000
        if len(audio) > max_audio_length:
            print(f"Truncating audio from {len(audio)} to {max_audio_length}")
            audio = audio[:max_audio_length]
        else:
            # Pad with zeros if shorter
            audio = np.pad(audio, (0, max_audio_length - len(audio)), "constant")

        # Normalize audio
        audio = (audio - np.mean(audio)) / (np.std(audio) + 1e-7)

        # Step 4: Convert text to phonemes using ConquiTTS
        phonemes = phonemizer.phonemize(batch["sentence"])
        print(f"Phonemes: {phonemes}")

        # Tokenize phonemes
        phoneme_ids = processor.tokenizer(
            phonemes,
            return_tensors="np",
            padding="max_length",
            truncation=True,
            max_length=processor.tokenizer.model_max_length,
        ).input_ids[0].astype(np.int32)  # Force smaller int32 type

        # Step 5: Process audio for Wav2Vec2
        input_values = processor(
            audio,
            sampling_rate=16000,
            return_tensors="np",
            padding="max_length",
            truncation=True,
            max_length=max_audio_length,
        ).input_values[0]

        # Debugging shapes
        print(f"Processed input_values shape: {input_values.shape}")
        print(f"Processed phoneme_ids shape: {phoneme_ids.shape}")

        # Assign processed data to batch
        batch["input_values"] = input_values
        batch["labels"] = phoneme_ids

    except Exception as e:
        print(f"Error processing batch: {e}")
        raise e

    return batch

# Load dataset
data = load_dataset("DTU54DL/common-accent")

# Preprocess dataset
try:
    data = data.map(preprocess_audio, remove_columns=["audio", "sentence"], batched=False)
except Exception as e:
    print(f"Dataset preprocessing failed: {e}")
    raise e

# Debug preprocessed data
print(data["train"][0])

# Define metric computation
def compute_metrics(pred):
    predictions = torch.argmax(torch.tensor(pred.predictions), axis=-1)
    label_ids = torch.tensor(pred.label_ids)
    accuracy = (predictions == label_ids).float().mean().item()
    return {"accuracy": accuracy}

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=processor)

# Training arguments
training_args = TrainingArguments(
    output_dir="./wav2vec2_common_accent_finetuned",
    run_name="wav2vec2_finetuning_run_1",
    overwrite_output_dir=True,
    eval_strategy="epoch",
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust based on GPU memory
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    report_to="none",
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [9]:
import torchaudio.datasets
print(dir(torchaudio.datasets))  # Check if "TIMIT" is listed


['CMUARCTIC', 'CMUDict', 'COMMONVOICE', 'DR_VCTK', 'FluentSpeechCommands', 'GTZAN', 'IEMOCAP', 'LIBRISPEECH', 'LIBRITTS', 'LJSPEECH', 'LibriLightLimited', 'LibriMix', 'LibriSpeechBiasing', 'MUSDB_HQ', 'QUESST14', 'SPEECHCOMMANDS', 'Snips', 'TEDLIUM', 'VCTK_092', 'VoxCeleb1Identification', 'VoxCeleb1Verification', 'YESNO', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'cmuarctic', 'cmudict', 'commonvoice', 'dr_vctk', 'fluentcommands', 'gtzan', 'iemocap', 'librilight_limited', 'librimix', 'librispeech', 'librispeech_biasing', 'libritts', 'ljspeech', 'musdb_hq', 'quesst14', 'snips', 'speechcommands', 'tedlium', 'utils', 'vctk', 'voxceleb1', 'yesno']


In [11]:
import torchaudio

# Load the LIBRISPEECH dataset
librispeech_train = torchaudio.datasets.LIBRISPEECH(
    root="./data",
    url="train-clean-100",  # Subset to load: "train-clean-100", "test-clean", etc.
    download=True,          # Download the dataset if not already available
)

# Inspect a single sample
waveform, sample_rate, transcript, speaker_id, chapter_id, utterance_id = librispeech_train[0]

print("Waveform shape:", waveform.shape)  # Shape: [number_of_samples]
print("Sample rate:", sample_rate)        # Typically 16,000 Hz
print("Transcript:", transcript)
print("Speaker ID:", speaker_id)
print("Chapter ID:", chapter_id)
print("Utterance ID:", utterance_id)


FileNotFoundError: [Errno 2] No such file or directory: './data/train-clean-100.tar.gz.5046f40e94624328bc9fe068592a29d0.partial'